# Coopang 광고 데이터 처리


## 함수화

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


def ad_blocker_coopang(keyword):

    options = Options()
    browser = webdriver.Chrome(options=options)

    base_url = "https://www.coupang.com/np/search?component"
    search_term = keyword

    browser.get(f"{base_url}=&q={search_term}&channel=user")

    soup = BeautifulSoup(browser.page_source, "html.parser")

    search_product_list = soup.find("ul", class_="search-product-list")

    filtered_products = [ product for product in search_product_list.find_all('li', recursive=False) if 'search-product__ad-badge' not in product['class'] ]


    results = []

    for i in range(len(filtered_products)):

        # brand_name, product_name
        s = filtered_products[i].select_one('.name').string
        split_result = s.split(' ', 1)  # 첫 번째 공백을 기준으로 문자열 분리
        brand_name = split_result[0]
        product_name = split_result[1]

        # link
        anchor = filtered_products[i].find('a')

        product_data = {

            'img_link' : 'https:'+filtered_products[i].select_one('.search-product-wrap-img')['src'],
            'brand' : brand_name,
            'name' : product_name.replace(",", ""),
            'price': int(filtered_products[i].select_one('.price-value').string.replace(",", "")),
            'link' : 'https://www.coupang.com/'+anchor['href']
        }

        results.append(product_data)
    
    return results

---

## csv 파일 처리

In [2]:
keyword = input('what do you want to search for? ')

products = ad_blocker_coopang(keyword)

# 입력 받은 키워드 이름으로 파일 오픈, 없으면 새로 생성
file = open(f"{keyword}.csv", "w",encoding="utf-8-sig")

# 파일에 헤더 입력 , \n
file.write("Image,Brand,Name,Price,URL\n")

# jobs 리스트, job dict
for product in products:
    file.write(f"{product['img_link']},{product['brand']},{product['name']},{product['price']},{product['link']}\n")

file.close()

what do you want to search for? 썬크림


---

## import 처리

- main.py 

In [3]:
from ad_blocker import ad_blocker_coopang

keyword = input('what do you want to search for? ')

products = ad_blocker_coopang(keyword)

# 입력 받은 키워드 이름으로 파일 오픈, 없으면 새로 생성
file = open(f"{keyword}.csv", "w",encoding="utf-8-sig")

# 파일에 헤더 입력 , \n
file.write("Image,Brand,Name,Price,URL\n")

# jobs 리스트, job dict
for product in products:
    file.write(f"{product['img_link']},{product['brand']},{product['name']},{product['price']},{product['link']}\n")

file.close()


what do you want to search for? 스프레이


---

# Flask

In [ ]:
from flask import Flask

# Flask 클래스의 인스턴스를 만들고, "Ad blocker"라는 이름을 부여하여 app 변수에 할당합니다
app = Flask("Ad blocker")

# 이 경로는 Flask 애플리케이션의 루트 경로입니다.
# home() 함수를 '/' 경로로 연결합니다. 

@app.route("/")
def home():
    return "hello flask"

# 모든 IP 허용 0.0.0.0 로컬 뿐만 아니라 외부 호스트에서도 Flask 애플리케이션에 접근할 수 있습니다.
app.run(host='0.0.0.0', port=8000)

## flask html return

- templates 폴더
- form 


<img src = "form.png">

In [8]:
from flask import Flask
from flask import render_template


# Flask 클래스의 인스턴스를 만들고, "Ad blocke"라는 이름을 부여하여 app 변수에 할당합니다
app = Flask("Ad blocker")

# 이 경로는 Flask 애플리케이션의 루트 경로입니다.
# home() 함수를 '/' 경로로 연결합니다. 

@app.route("/")
def home():
    return render_template('home.html')


# /search 루트 'http://192.168.45.143:8000/search?keyword=썬크림' 

@app.route('/search')
def search():
    return 'form action'

# 모든 IP 허용 0.0.0.0 로컬 뿐만 아니라 외부 호스트에서도 Flask 애플리케이션에 접근할 수 있습니다.
app.run(host='0.0.0.0', port=8000)

 * Serving Flask app 'Ad blocker'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://192.168.45.143:8000
Press CTRL+C to quit
192.168.45.143 - - [06/Jun/2023 14:41:05] "GET / HTTP/1.1" 200 -
192.168.45.143 - - [06/Jun/2023 14:41:05] "GET /apple-touch-icon-precomposed.png HTTP/1.1" 404 -
192.168.45.143 - - [06/Jun/2023 14:41:05] "GET /apple-touch-icon.png HTTP/1.1" 404 -
192.168.45.143 - - [06/Jun/2023 14:41:05] "GET /search?keyword=썬크림 HTTP/1.1" 200 -
192.168.45.143 - - [06/Jun/2023 14:41:05] "GET /apple-touch-icon-precomposed.png HTTP/1.1" 404 -
192.168.45.143 - - [06/Jun/2023 14:41:05] "GET /apple-touch-icon.png HTTP/1.1" 404 -
192.168.45.143 - - [06/Jun/2023 14:41:05] "GET /favicon.ico HTTP/1.1" 404 -
192.168.45.143 - - [06/Jun/2023 14:41:05] "GET /apple-touch-icon-precomposed.png HTTP/1.1" 404 -
192.168.45.143 - - [06/Jun/2023 14:41:05] "GET /apple-touch-icon.png HTTP/1.1" 404 -
192.168.45.143 - - [06/Jun/2023 14:41:05] "GET /favicon.ico HTTP/1.1" 404 -
192.168.45.143 - -

---

## flask request

- args

In [ ]:
from flask import Flask
from flask import render_template
from flask import request

from ad_blocker import ad_blocker_coopang


# Flask 클래스의 인스턴스를 만들고, "Ad blocke"라는 이름을 부여하여 app 변수에 할당합니다
app = Flask("Ad blocker")

# 이 경로는 Flask 애플리케이션의 루트 경로입니다.
# home() 함수를 '/' 경로로 연결합니다. 

@app.route("/")
def home():
    return render_template('home.html')


# /search 루트 'http://192.168.45.143:8000/search?keyword=썬크림' 

@app.route('/search')
def search():
    
    keyword = request.args.get('keyword')     # keyword = request.args['keyword']
    
    products = ad_blocker_coopang(keyword)
    
    return render_template('search.html',keyword = keyword, products=products)

# 모든 IP 허용 0.0.0.0 로컬 뿐만 아니라 외부 호스트에서도 Flask 애플리케이션에 접근할 수 있습니다.
app.run(host='0.0.0.0', port=8000)

 * Serving Flask app 'Ad blocker'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://192.168.45.143:8000
Press CTRL+C to quit
192.168.45.143 - - [06/Jun/2023 14:52:32] "GET / HTTP/1.1" 200 -
192.168.45.143 - - [06/Jun/2023 14:52:41] "GET /search?keyword=썬크림 HTTP/1.1" 200 -
